In [15]:
import keras
from keras.models import load_model
import random
import numpy as np
import math
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from IPython.display import clear_output
import time
import pandas_datareader as pdr
import pandas_datareader.data as web
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf

In [16]:
def formatPrice(n):
	return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

In [20]:
def rsi(data,t):
    up=0
    down=0
    window_length = 14
    close = data['Adj Close']
    delta = close.diff()
    for i in range(t-window_length+1,t):
        if delta[i] >0:
            up=up+delta[i]
        else:
            down=down+abs(delta[i])
            
    up=up/window_length
    down=down/window_length
    RS = up / down
    RSI = 100.0 - (100.0 / (1.0 + RS))
    return RSI

In [25]:
class Portfolio:
    def __init__(self):
        self.cash=1000000
        self.profit=0
        self.inventory = []
        
    def getData(self):        
        self.vec = []
        self.start = datetime(2007, 1, 1)
        self.end = datetime(2019, 3, 3)
        self.df = pd.read_csv('/home/admin-pc/Desktop/19march/5data/nifty50.csv')[1700:]
        return self.df 
    
    def getState(self,data,t):
        self.n=50
        self.res=[]
        for i in range(self.n):
            if ((data['Close'][t-i]-data['Close'][t-i-1])/data['Close'][t-i-1])*100 > 0.25:
                a=1
            elif ((data['Close'][t-i]-data['Close'][t-i-1])/data['Close'][t-i-1])*100 < -0.25:
                a=-1
            else:
                a=0          
            self.res.append(a)
        self.res.append(Portfolio().cash)
        self.res.append(Portfolio().profit)
        return np.array([self.res])


In [26]:
class RLAgent:
    def __init__(self, model_name=""):
        self.state_size = 52 # normalized previous days
        self.action_size = 3 # sit, buy, sell
        self.memory = deque(maxlen=1000)
        self.nshare=0
        self.last_price=0
        self.model_name = model_name
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = load_model("models/model_ep95")


    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)

        options = self.model.predict(state)
        return np.argmax(options[0])

In [30]:
#************************-Testing-*********************

rl = RLAgent()

agent=Portfolio()
data=agent.getData()
print(data.shape[0])
data['Action']=""
data['actual_action']=""
data['share']=""
data['Cash']=""
data['trade_profit']=""
data['Adj_cash']=""
state=agent.getState(data,51)

l = len(data) - 1
batch_size = 32
state = agent.getState(data, 51)
initial_cash=agent.cash
total_profit = 0
agent.nshare=0
agent.last_price=0
data['Action'][50]=0
data['share'][50]=agent.nshare
data['Cash'][50]= agent.cash

for t in range(51,l):
    action = rl.act(state)
    data['Action'][t]=0
    data['share'][t]=agent.nshare
    data['Cash'][t]= agent.cash


    # sit
    next_state = agent.getState(data, t + 1)
    reward = 0

    if action == 1:# buy
            data['Action'][t]=1
            trend=rsi(data,t)
            if trend <= 30:                            
                if  agent.nshare == 0: 
                    agent.nshare=int(agent.cash/data['Close'][t])
                    agent.cash=agent.cash-agent.nshare*data['Close'][t]
                    agent.last_price=data['Close'][t]
                    data['actual_action'][t]="Bought"

                elif agent.nshare < 0: 
                    sell_price=agent.last_price
                    ns = agent.nshare
                    agent.nshare=0
                    data['trade_profit'][t]=(sell_price - data['Close'][t])*(-ns)
                    total_profit = total_profit + (sell_price - data['Close'][t])*(-ns)
                    agent.cash=agent.cash+(-ns*sell_price)+data['trade_profit'][t]
                    data['actual_action'][t]="Bought"

                data['share'][t]=agent.nshare
                data['Cash'][t]= agent.cash
                data['Adj_cash'][t]=agent.cash+agent.nshare*data['Close'][t]
                reward = (1+((data['Close'][t]-data['Close'][t-1])/data['Close'][t-1])*(data['Close'][t-1]/data['Close'][t-50]))

    elif action == 2:# sell
        data['Action'][t]=2
        trend=rsi(data,t)
        if trend >=  70:  
                if  agent.nshare == 0: 
                    agent.nshare=int(agent.cash/data['Close'][t])

                    agent.cash=agent.cash-agent.nshare*data['Close'][t]
                    agent.nshare= -agent.nshare
                    agent.last_price=data['Close'][t]
                    data['actual_action'][t]="Sold"

                elif agent.nshare > 0: 
                    buy_price=agent.last_price
                    ns = agent.nshare
                    agent.nshare=0
                    data['trade_profit'][t]=(data['Close'][t]-buy_price)*(ns)
                    total_profit = total_profit + (data['Close'][t]-buy_price)*(ns)
                    agent.cash=agent.cash+(ns)*data['Close'][t]
                    data['actual_action'][t]="Sold"

                data['share'][t]=agent.nshare
                data['Cash'][t]= agent.cash
                data['Adj_cash'][t]=agent.cash+(-agent.nshare)*agent.last_price+(agent.last_price - data['Close'][t])*(-agent.nshare)
                reward = -1*(1+((data['Close'][t]-data['Close'][t-1])/data['Close'][t-1])*(data['Close'][t-1]/data['Close'][t-50]))

    done = True if t == l - 1 else False
    rl.memory.append((state, action, reward, next_state, done))
    state = next_state

    if done:

            print( "------------------------------------------")
            print ("Initial Cash: " + formatPrice(initial_cash))
            print ("Total  Profit: " + formatPrice(total_profit))
            print ("Final cash: " + formatPrice(agent.cash))
            print ("final holding: {} ".format(agent.nshare))
            print("-------------------------------------------")


------------------------------------------
Initial Cash: $1000000.00
Total  Profit: $324000.00
Final cash: $4123.00
final holding: 312 
-------------------------------------------
